# Causal Discovery & ML - From Assumptions to Applications

### Loading Libraries

In [6]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# IterTools
from itertools import combinations

# Statistical Models
from scipy import stats
import statsmodels.api as sm

# Networkx
import networkx as nx

# JavaScript Object Notation
import json

# Data Visualization
import graphviz
import seaborn as sns
import matplotlib.pyplot as plt

# Scikit-Learn
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, LassoCV
from sklearn.metrics import mean_absolute_percentage_error

# Castle
import castle
from castle.common import GraphDAG
from castle.metrics import MetricsDAG
from castle.datasets import DAG, IIDSimulation 

from castle.algorithms import PC, GES
from castle.algorithms import ANMNonlinear, ICALiNGAM, DirectLiNGAM
from castle.algorithms import Notears, NotearsNonlinear, GOLEM

from castle.common.priori_knowledge import PrioriKnowledge

from castle.common.independence_tests import hsic_test

# PyTorch & Transformers
import torch

# DoWhy Causal Libraries
import dowhy
from dowhy import gcm
from dowhy import CausalModel
from dowhy.causal_model import CausalModel

# Notebook Iteration 
from tqdm import tqdm

# Operating Systems & Environments
import os
os.environ['CASTLE_BACKEND'] = 'pytorch'

# Copy
from copy import deepcopy

# Light Gradient Boosting Models
from lightgbm import LGBMRegressor, LGBMClassifier

#### Setting Viz Standards

In [7]:
COLORS = [
    '#00B0F0',
    '#FF0000',
    '#B0F000'
]

#### Setting Seeds

In [8]:
# Set random seed
SEED = 18

np.random.seed(SEED)

## Introduction to gCastle

### Synthetic Data in gCastle

In [10]:
# Generate a scale-free adjacency matrix
adj_matrix = DAG.scale_free(
    n_nodes=10,
    n_edges=17,
    seed=SEED
)

In [ ]:
adj_matrix

In [ ]:
# Visualize the adjacency matrix
g = nx.DiGraph(adj_matrix)

plt.figure(figsize=(12, 8))
nx.draw(
    G=g,
    node_color=COLORS[0],
    node_size=1200,
    arrowsize=17,
    with_labels=True,
    font_color='white',
    font_size=21,
    pos=nx.circular_layout(g)
)

In [ ]:
dataset = IIDSimulation(
    W=adj_matrix, 
    n=10000, 
    method='linear', 
    sem_type='gauss'
)

In [ ]:
# Access the generated data
dataset.X

### Fitting a First Causal Discovery Model

In [ ]:
# Instantiate the model
pc = PC()

In [ ]:
# Fit the model
pc.learn(dataset.X)

In [ ]:
pred_dag = pc.causal_matrix
pred_dag

#### Visualizing The Model

In [ ]:
g_pred = nx.DiGraph(pred_dag)

plt.figure(figsize=(12, 8))
nx.draw(
    G=g_pred,
    node_color=COLORS[0],
    node_size=1200,
    arrowsize=17,
    with_labels=True,
    font_color='white',
    font_size=21,
    pos=nx.circular_layout(g)
)

In [ ]:
GraphDAG(
    est_dag=pred_dag, 
    true_dag=adj_matrix)

plt.show()

#### Model Evaluation Metrics

In [ ]:
metrics = MetricsDAG(
    B_est=pred_dag, 
    B_true=adj_matrix)

In [ ]:
metrics.metrics['F1']

In [ ]:
metrics.metrics

In [ ]:
	
def get_n_undirected(g):
    
    total = 0
    
    for i in range(g.shape[0]):
        for j in range(g.shape[0]):
            if (g[i, j] == 1) and (g[i, j] == g[j, i]):
                total += .5
    
    return total

In [ ]:
get_n_undirected(pred_dag)

In [ ]:
np.tril(pred_dag)

### Constraint-Based Causal Discovery

In [ ]:
# Build a DAG adj matrix
pc_dag = np.array([
    [0, 0, 1, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 0, 0, 0]
])

In [ ]:
# Generate data from this DAG
N = 1000

p = np.random.randn(N)
q = np.random.randn(N)

r = p + q + .1 * np.random.randn(N)
s = .7 * r + .1 * np.random.randn(N)

# To matrix
pc_dataset = np.vstack([p, q, r, s]).T

In [ ]:
# Plot the original graph

# Get learned graph
true_graph = nx.DiGraph(pc_dag)

# Relabel the nodes
MAPPING = {k: v for k, v in zip(range(4), ['P', 'Q', 'R', 'S'])}
true_graph = nx.relabel_nodes(true_graph, MAPPING, copy=True)

plt.figure(figsize=(7, 3))
nx.draw(
    G=true_graph,
    node_color=COLORS[0],
    node_size=1200,
    arrowsize=17,
    with_labels=True,
    font_color='white',
    font_size=21,
    pos=nx.circular_layout(true_graph)
)

### PC

In [ ]:
# Initialize PC
pc = PC()

# Fit 
pc.learn(pc_dataset)

# Display the learned matrix
pc.causal_matrix

In [ ]:
# Visualize
GraphDAG(
    est_dag=pc.causal_matrix, 
    true_dag=pc_dag)

plt.show()

In [ ]:
# Get metrics
MetricsDAG(
    B_est=pc.causal_matrix,
    B_true=pc_dag
).metrics

### PC-Stable

In [2]:
# PC-stable
pc_stable = PC(variant='stable')

# Fit 
pc_stable.learn(pc_dataset)

# Visualize
GraphDAG(
    est_dag=pc_stable.causal_matrix, 
    true_dag=pc_dag)

plt.show()

### PC-Parallel

In [ ]:
# PC-stable
pc_parallel = PC(variant='parallel')

# Fit 
pc_parallel.learn(pc_dataset)

# Visualize
GraphDAG(
    est_dag=pc_parallel.causal_matrix, 
    true_dag=pc_dag)

plt.show()

### PC with Categorical Data

In [ ]:
# Generate the data
a = np.random.binomial(4, .5, N)
b = np.random.binomial(4, .5, N)
c = ((a + b + np.random.normal(0, 1, N)) > 2).astype(int)

# To matrix
pc_cat_dataset = np.vstack([a, b, c]).T

In [ ]:
pc_cat = PC(ci_test='chi2')

In [ ]:
pc_cat.learn(pc_cat_dataset)

In [ ]:
pc_cat.causal_matrix

### PC with Alternative Independence Tests

In [ ]:
from castle.common.independence_tests import CITest

In [ ]:
pc_cat_alt = PC(ci_test=CITest.cressie_read)

In [ ]:
pc_cat_alt.learn(pc_cat_dataset)

In [ ]:
pc_cat_alt.causal_matrix

### Score-Based Causal Discovery